In [6]:
from light_curves import LightCurve
from exposures import Calexp
from task import Run
import numpy as np
import matplotlib.pyplot as plt
import random
import json
from tqdm.notebook import tqdm

In [5]:
radius = 0.2 #[0.1,0.2,0.3, 0.5]
density = 5/0.05#]np.array([10,50, 100, 500])/0.05
n_max_calexps = 100
bands = "gri"
problems = []
delta_mag = np.array([2.5,2,1.5,1.0,0.5,0])[-len(bands):]
print(delta_mag)


process = Run(ra=57.59451632893858, dec=-32.481152201226145, 
              density = density, scale = radius, bands = bands, 
              name = f"run_{density:.0f}dens_{int(radius*10):02d}rad_{bands}_{n_max_calexps}calexps", 
              calexps_method="overlap", measure_method ="ForcedMeas")
# process.measure_method = "SingleFrame"
print(process)
process.inject_task()
schema = process.measure_task()

process.collect_calexp()
process.data_calexp.to_csv(process.main_path+'/data_calexp.csv', index=False)
for event_id, m in enumerate(tqdm(np.linspace(17,22,process.n_lc), desc="Loading events:")):
    lc_ra, lc_dec = process.generate_location()
    params = {"t_0": random.uniform(60300,61500),
   "t_E": random.uniform(20, 200), 
   "u_0": random.uniform(0.1,1)}
    for band, dm in zip(process.bands, delta_mag):
        params["m_base"]=m+dm
        process.add_lc(lc_ra, lc_dec, params, event_id=event_id, band=band)
process.data_events.to_csv(process.main_path+'/data_events.csv', index=False) 
# process.sky_map()
process.log_task("Add and simulate light curves")


for idx, data in process.data_calexp.iterrows():
    if idx >= n_max_calexps:
        break
    print(f" ------ CALEXP {idx+1}/{n_max_calexps} ------")
    data_id = data[['detector', 'visit']].to_dict()
    calexp = Calexp(data_id)
    if calexp.overlaps(process.region):
        band = data["band"]; print("Band: ", band)
        # ax = calexp.plot() 
        # for lc in process.inj_lc:
        #     calexp.add_point(ax, lc.ra, lc.dec, r=40)
        # plt.show()
        process.data_calexp.loc[(process.data_calexp["detector"] == data_id["detector"]) & (process.data_calexp["visit"] == data_id["visit"]),"overlap"] = True
        injection_catalog = process.create_injection_table(calexp, band)
        injection_catalog_checked = process.check_injection_catalog(calexp, catalog = injection_catalog)
        
        # print(f"{injection_catalog_checked=}")
        if injection_catalog_checked:
            print(f"{band=}")
            injected_exposure, injected_catalog = process.inject_calexp(calexp, injection_catalog_checked, save_fit=f"calexp_{idx}_{band}.fit")
            if injected_catalog != None:
                inj_calexp = Calexp(injected_exposure)
                # ax = inj_calexp.plot() 
                #     if lcc["injection_flag"] != 0:
                #         color = "cyan"
                #     else:
                #         color = "red"
                #     inj_calexp.add_point(ax, lcc["ra"], lcc["dec"], r=40, c=color)
                # plt.show()
                injected_catalog_checked = process.check_injection_catalog(calexp, injected_catalog, before_injection=False)
                if len(injected_catalog_checked)>0:
                    table, sources = process.measurement(schema, inj_calexp, injected_catalog)
                    for _, lc_inj in table.iterrows():
                        filtered_idx = process.data_events[
                            (process.data_events["ra"] == lc_inj["ra"]) &
                            (process.data_events["dec"] == lc_inj["dec"]) &
                            (process.data_events["band"] == band)].index
                        id_lc_inj = filtered_idx[0] if not filtered_idx.empty else None
                        lc = process.inj_lc[id_lc_inj]
                        flux, flux_err, mag, mag_err = table.loc[(table["ra"] == lc.ra) & (table["dec"] == lc.dec), ["flux", "flux_err", "mag", "mag_err"]].values[0]
                        lc.add_data(data_id, [flux, flux_err, mag, mag_err])
                        # print("-"*50) ["mjd", "detector", "visit", "flux", "flux_err", "mag", "mag_err"]
                        # print(f"ra = {lc.ra}, dec = {lc.dec}, id = {id_lc_inj}, band = {lc.band}")
                        # print("Measured ", mag, mag_err)
                        # print("Injected", lc.data["mag_sim"][idx])
                        # is_within_range = (mag - mag_err <= lc.data["mag_sim"][idx] <= mag + mag_err)
                        # print("Injected within range:", is_within_range)
                        # print("Injected inj_catalog ", lc_inj["mag"])
                        # try:
                        #     if abs(mag-lc.data["mag_inj"][idx])>1:
                        #         problems.append([idx, id_lc_inj, lc.ra, lc.dec])
                        #         # problems.append((idx, calexp, process.inj_lc[id_lc_inj], lc.ra , lc.dec, sources, injected_catalog, injected_catalog_checked, mag, mag_err, lc.data["mag_inj"][idx], injected_exposure))
                        # except Exception as e:
                        #     print(f"flag: {e}")

        else:
            print("Injection catalog is empty")
        # print(f"{len(problems)}")
    else:
        print("Not intersection between calexp and region.")
            
process.data_calexp.to_csv(process.main_path+'/data_calexp.csv', index=False)
process.data_events.to_csv(process.main_path+'/data_events.csv', index=False)  
process.save_lc()
process.save_time_log()
process.time_analysis()
print("Injected points per light curve", [lc.data["mag"].count() for lc in process.inj_lc] )
process.sky_map(calexps=n_max_calexps)
for band in process.bands:
    process.sky_map(calexps=n_max_calexps, band=band)

[1.  0.5 0. ]
----------------------------------------
Run Name: run_100dens_02rad_gri_100calexps
Center: RA=57.59451632893858, Dec=-32.481152201226145
Band: gri
Calexp Method: overlap
Measure Method: SingleFrame
Density: 100.0 sources/deg²
Area: 0.126 deg²
Scale: 0.200 deg
Number of LightCurves to inject: 13
Main Path: ./runs/run_100dens_02rad_gri_100calexps
----------------------------------------
 ------ CALEXP 1/100 ------
Band:  r


IndexError: index 0 is out of bounds for axis 0 with size 0

In [7]:
from tqdm.notebook import tqdm
for band in process.bands:
    lcs = [lc for lc in process.inj_lc if lc.band == band]
    print(f"Saving {len(lcs)} light curves in band {band}")
    for i, lc in enumerate(tqdm(lcs)):
        event_id = process.data_events["event_id"][i]
        lc.save(process.main_path+f"/lc_{event_id}_{lc.band}.csv")

Saving 283 light curves in band u


  0%|          | 0/283 [00:00<?, ?it/s]

Saving 283 light curves in band g


  0%|          | 0/283 [00:00<?, ?it/s]

Saving 283 light curves in band r


...

  0%|          | 0/283 [00:00<?, ?it/s]

Saving 283 light curves in band z


  0%|          | 0/283 [00:00<?, ?it/s]

Saving 283 light curves in band y


  0%|          | 0/283 [00:00<?, ?it/s]